In [ ]:
# This is Final Exam for IBM Time Series course.
# I use monthly vw market data.
# Not everything works here =(

In [ ]:
!pip install statsmodels==0.11.0
!pip install pmdarima

In [ ]:
from datetime import datetime
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
plt.style.use('seaborn-white')
from datetime import datetime
import tensorflow as tf
import warnings
warnings.simplefilter(action='ignore')
import keras
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM, Activation, Dropout
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

print(sm.__version__)

In [ ]:
path = '../input/monthly-market-returns/mMKT.CSV'
df = pd.read_csv(path)
df.shape

In [ ]:
df.head()

In [ ]:
df.date=pd.to_datetime(df.date,format='%Y%m')
df.date=df.date.dt.strftime("%Y-%m")

In [ ]:
df.set_index('date', inplace=True)
df.info()

In [ ]:
df.describe()
sns.lineplot(x=df.index, y='mkt', data=df)

In [ ]:
# testing stationarity and seasonality
import statsmodels.tsa.stattools as ts

results = ts.adfuller(df)
results
# ts is stationary.

In [ ]:
# forecasting using ARIMA

# first, see acf/pacf

sm.tsa.graphics.plot_acf(df.mkt, lags=24, zero=False);

In [ ]:
sm.tsa.graphics.plot_pacf(df.mkt, lags=24, zero=False);

In [ ]:
# acf and pacf look like noise, not sure which arma to use.

from pmdarima.arima import auto_arima

train = df[0:900]
test = df[900:]

In [ ]:
# try to autodetect arima order, use this: https://towardsdatascience.com/time-series-forecasting-using-auto-arima-in-python-bb83e49210cd

arima_model = auto_arima(train, start_p=0, start_q=1, d=0, max_p=12, max_d=0, max_q=12, seasonal=False, n_fits=50, steepwise=False)
arima_model

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

import warnings
warnings.filterwarnings("ignore")

predictions = []
actual = []

#len(df.mkt)

for i in np.arange(10, len(df.mkt)):
    arima_model = ARIMA(df.mkt[0:i], order=(0,0,1))
    arima_model_fit = arima_model.fit(disp=0)
    yhat = arima_model_fit.forecast()
    predictions.append(yhat[0])
    actual.append(df.mkt[i])
    
warnings.resetwarnings()


In [ ]:
predictions = np.concatenate(predictions, axis=0)
print(predictions[0:10])
print(actual[0:10])
np.mean((actual-predictions)**2)
np.mean((actual-predictions)**2)

In [ ]:
print(len(predictions))
print(len(actual))
print(np.mean((actual-predictions)**2))
print(np.mean((actual-np.full((len(actual)), 0.5))**2))

In [ ]:
# define functions, needed for RNN forecasting

def get_keras_format_series(series):
    """Convert a series to a numpy array of shape [n_obs, time_steps, features]"""
    
    series = np.array(series)
    return series.reshape(series.shape[0], series.shape[1], 1)

def get_train_test_data(df, series_name, input_periods, test_periods):
    """
    Utility processing function that splits an hourly time series into 
    train and test with keras-friendly format, according to user-specified
    choice of shape.    
    
    arguments
    ---------
    df (dataframe): dataframe with time series columns
    series_name (string): column name in df
    input_hours (int): length of sequence input to network 
    test_hours (int): length of held-out terminal sequence
    
    returns
    ---------
    tuple: train_X, test_X_init, train_y, test_y     
    """
    
    forecast_series = df[series_name]
    train = forecast_series[:-test_periods] 
    test = forecast_series[-test_periods:] 

    train_X, train_y = [], []

    # range 0 through # of train samples - input_hours # This is to create many samples with corresponding
    for i in range(0, train.shape[0]-input_periods): 
        train_X.append(train[i:i+input_periods]) # each training sample is of length input hours
        train_y.append(train[i+input_periods]) # each y is just the next step after training sample

    train_X = get_keras_format_series(train_X) # format our new training set to keras format
    train_y = np.array(train_y) # make sure y is an array to work properly with keras
    
    # The set that we had held out for testing (must be same length as original train input)
    test_X_init = test[:input_periods] 
    test_y = test[input_periods:] # test_y is remaining values from test set
    
    return train_X, test_X_init, train_y, test_y

In [ ]:
# forecasting using RNN

train_X, test_X_init, train_y, test_y = get_train_test_data(df, 'mkt', 12, 24)
print('Training input shape: {}'.format(train_X.shape))
print('Training output shape: {}'.format(train_y.shape))
print('Test input shape: {}'.format(test_X_init.shape))
print('Test output shape: {}'.format(test_y.shape))

In [ ]:
def fit_SimpleRNN(train_X, train_y, cell_units, epochs):
    """
    Fit Simple RNN to data train_X, train_y 
    
    arguments
    ---------
    train_X (array): input sequence samples for training 
    train_y (list): next step in sequence targets
    cell_units (int): number of hidden units for RNN cells  
    epochs (int): number of training epochs   
    """

    # initialize model
    model = Sequential() 
    
    # construct an RNN layer with specified number of hidden units
    # per cell and desired sequence input format 
    model.add(SimpleRNN(cell_units, input_shape=(train_X.shape[1],1)))
    
    # add an output layer to make final predictions 
    model.add(Dense(1))
    
    # define the loss function / optimization strategy, and fit
    # the model with the desired number of passes over the data (epochs) 
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(train_X, train_y, epochs=epochs, batch_size=64, verbose=0)
    
    return model

In [ ]:
model_rnn = fit_SimpleRNN(train_X, train_y, cell_units=10, epochs=300)
# model.summary()

In [ ]:
def predict(X_init, n_steps, model):
    """
    Given an input series matching the model's expected format,
    generates model's predictions for next n_steps in the series      
    """
    
    #X_init = X_init.copy().reshape(1,-1,1)
    #X_init = np.array(X_init)
    X_init = np.array(test_X_init.copy())
    X_init = X_init.reshape(1,12,1)

    preds = []
    
    # iteratively take current input sequence, generate next step pred,
    # and shift input sequence forward by a step (to end with latest pred).
    # collect preds as we go.
    for _ in range(n_steps):
        pred = model.predict(X_init)
        preds.append(pred)
        X_init[:,:-1,:] = X_init[:,1:,:] # replace first 11 values with 2nd through 12th
        X_init[:,-1,:] = pred # replace 12th value with prediction
    
    preds = np.array(preds).reshape(-1,1)
    
    return preds

def predict_and_plot(X_init, y, model, title):
    """
    Given an input series matching the model's expected format,
    generates model's predictions for next n_steps in the series,
    and plots these predictions against the ground truth for those steps 
    
    arguments
    ---------
    X_init (array): initial sequence, must match model's input shape
    y (array): true sequence values to predict, follow X_init
    model (keras.models.Sequential): trained neural network
    title (string): plot title   
    """
    
    y_preds = predict(test_X_init, n_steps=len(y), model=model) # predict through length of y
    # Below ranges are to set x-axes
    start_range = range(1, test_X_init.shape[0]+1) #starting at one through to length of test_X_init to plot X_init
    predict_range = range(test_X_init.shape[0], 24)  #predict range is going to be from end of X_init to length of test_hours
    
    #using our ranges we plot X_init
    plt.plot(start_range, test_X_init)
    #and test and actual preds
    plt.plot(predict_range, test_y, color='orange')
    plt.plot(predict_range, y_preds, color='teal', linestyle='--')
    
    plt.title(title)
    plt.legend(['Initial Series','Target Series','Predictions'])

In [ ]:
tX = np.array(test_X_init.copy())
tX=tX.reshape(1,12,1)
tX.shape

In [ ]:
model_rnn.predict(np.array(tX))

In [ ]:
#predict(test_X_init, 12, model_rnn)
predict_and_plot(test_X_init, test_y, model_rnn, 'PM Series: Test Data and Simple RNN Predictions')

In [ ]:
predictions_rnn = predict(test_X_init, 12, model_rnn)
print(predictions_rnn)
print(test_y)
print(np.mean((np.array(test_y)-predictions_rnn)**2))
print(np.mean((np.array(test_y)-0.5)**2))

In [ ]:
# forecasting using LSTM



In [ ]:
def fit_LSTM(train_X, train_y, cell_units, epochs):
    """
    Fit LSTM to data train_X, train_y 
    
    arguments
    ---------
    train_X (array): input sequence samples for training 
    train_y (list): next step in sequence targets
    cell_units (int): number of hidden units for LSTM cells  
    epochs (int): number of training epochs   
    """
    
    # initialize model
    model = Sequential() 
    
    # construct a LSTM layer with specified number of hidden units
    # per cell and desired sequence input format 
    model.add(LSTM(cell_units, input_shape=(train_X.shape[1],1))) #,return_sequences= True))
    #model.add(LSTM(cell_units_l2, input_shape=(train_X.shape[1],1)))
    
    # add an output layer to make final predictions 
    model.add(Dense(1))
    
    # define the loss function / optimization strategy, and fit
    # the model with the desired number of passes over the data (epochs) 
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(train_X, train_y, epochs=epochs, batch_size=64, verbose=0)
    
    return model

In [ ]:
model_lstm = fit_LSTM(train_X, train_y, cell_units=30, epochs=1500) 

predict_and_plot(test_X_init, test_y, model_lstm, 
                 'PM_Nongzhanguan Series: Test Data and LSTM Predictions')

predictions_lstm = predict(test_X_init, 12, model_lstm)
#print(predictions_rnn)
#print(test_y)
print(np.mean((np.array(test_y)-predictions_lstm)**2))
print(np.mean((np.array(test_y)-0.5)**2))

In [ ]:
# forecasting using transformer?